# Implementacja RSA

Autorzy:

- Jakub Błażejowski | 145393

In [ ]:
# requires-python = ">3.8.0"

# python requirements

"""
sympy==1.11.1
"""

import math
import itertools
import random
from sympy import randprime
from IPython.display import Markdown
from ipywidgets import Button, Checkbox, IntText, Output

In [ ]:
%%html
<style>
    .output {
        align-items: center;
    }
    div.output_subarea {
        max-width: none;
        display: flex;
        justify-content: center;
    }
    :root {
        --jp-widgets-inline-label-width: 200px;
        --jp-widgets-inline-width: 408px;
    }
</style>

In [ ]:
class KeyGenerator:
    def __init__(
        self,
        prime1_widget: IntText,
        generate_prime1_widget: Checkbox,
        prime2_widget: IntText,
        generate_prime2_widget: Checkbox,
        start_widget: Button,
        output_widget: Output
    ):
        self.prime1_widget = prime1_widget
        self.generate_prime1_widget = generate_prime1_widget
        self.prime2_widget = prime2_widget
        self.generate_prime2_widget = generate_prime2_widget
        self.start_widget = start_widget
        self.output_widget = output_widget
        self.shared_key_part = 0
        self.public_key_part = 0
        self.private_key_part = 0
        
        self.widgets = [
            prime1_widget,
            generate_prime1_widget,
            prime2_widget,
            generate_prime2_widget,
            start_widget
        ]
        
        self.generate_keys = output_widget.capture()(self.generate_keys)
        
    def disable_widgets(self):
        for widget in self.widgets:
            widget.enabled = False
            
    def enable_widgets(self):
        for widget in self.widgets:
            widget.enabled = True
            
    def generate_keys(self):
        self.output_widget.clear_output()
        self.disable_widgets()
        self.start_widget.description = "Computing"
        
        prime1 = self.prime1_widget.value
        prime2 = self.prime2_widget.value
        
        if self.generate_prime1_widget.value:
            prime1 = randprime(1000, 10000)
            self.prime1_widget.value = prime1
            
        if self.generate_prime2_widget.value:
            prime2 = randprime(1000, 10000)
            self.prime2_widget.value = prime2
            
        self.shared_key_part = prime1 * prime2
        print("Shared key part:  ", self.shared_key_part)
        
        phi = (prime1 - 1) * (prime2 - 1)
        
        for i in itertools.count(start=2):
            if math.gcd(i, phi) == 1:
                self.public_key_part = i
                break
                
        print("Public key part:  ", self.public_key_part)
                
        for i in itertools.count(start=1):
            value = i * phi + 1
            
            if value%self.public_key_part == 0:
                self.private_key_part = value // self.public_key_part
                break
                
        print("Private key part: ", self.private_key_part)
        
        
        self.start_widget.description = "Start"
        self.enable_widgets()

In [ ]:
class Processor:
    def __init__(
        self,
        message_widget: IntText,
        generate_message_widget: Checkbox, 
        modulus_widget: IntText,
        power_widget: IntText,
        start_widget: Button,
        output_widget: Output
    ):
        self.message_widget = message_widget
        self.generate_message_widget = generate_message_widget
        self.modulus_widget = modulus_widget
        self.power_widget = power_widget
        self.start_widget = start_widget
        self.output_widget = output_widget
        self.result = 0
        
        self.widgets = [
            message_widget,
            generate_message_widget,
            modulus_widget,
            power_widget,
            start_widget
        ]
        
        self.process = output_widget.capture()(self.process)
        
    def disable_widgets(self):
        for widget in self.widgets:
            widget.enabled = False
            
    def enable_widgets(self):
        for widget in self.widgets:
            widget.enabled = True
        
    def process(self):
        self.output_widget.clear_output()
        self.disable_widgets()
        self.start_widget.description = "Computing"
        
        message = self.message_widget.value
        modulus = self.modulus_widget.value
        power = self.power_widget.value
        
        if self.generate_message_widget.value:
            message = random.randint(100000, 999999)
            self.message_widget.value = message
        
        self.result = pow(message, power, mod=modulus)
        print("Result: ", self.result)
        
        self.start_widget.description = "Start"
        self.enable_widgets()

In [ ]:
prime1_widget = IntText(description="1st prime:")
generate_prime1_widget = Checkbox(value=True, description="Generate 1st prime")
prime2_widget = IntText(description="2nd prime:")
generate_prime2_widget = Checkbox(value=True, description="Generate 2nd prime")
start_widget = Button(description="Start")
output_widget = Output()

key_generator = KeyGenerator(
    prime1_widget,
    generate_prime1_widget,
    prime2_widget,
    generate_prime2_widget,
    start_widget,
    output_widget
)

start_widget.on_click(lambda _: key_generator.generate_keys())

display(
    prime1_widget,
    generate_prime1_widget,
    prime2_widget,
    generate_prime2_widget,
    start_widget,
    output_widget
)

In [ ]:
decrypted_widget = IntText(description="Decrypted:", readout_format="50f")
encrypted_widget = IntText(description="Encrypted:")
generate_decrypted_widget = Checkbox(value=True, description="Generate decrypted")
generate_encrypted_widget = Checkbox(description="Generate encrypted")
shared_key_part_widget = IntText(value=key_generator.shared_key_part, description="Shared key part:")
public_key_part_widget = IntText(value=key_generator.public_key_part, description="Public key part:")
private_key_part_widget = IntText(value=key_generator.private_key_part, description="Private key part:")
start_encryption_widget = Button(description="Start")
start_decryption_widget = Button(description="Start")
encryption_output_widget = Output()
decryption_output_widget = Output()

encryptor = Processor(
    decrypted_widget,
    generate_decrypted_widget,
    shared_key_part_widget,
    public_key_part_widget,
    start_encryption_widget,
    encryption_output_widget
)

decryptor = Processor(
    encrypted_widget,
    generate_encrypted_widget,
    shared_key_part_widget,
    private_key_part_widget,
    start_decryption_widget,
    decryption_output_widget
)

start_encryption_widget.on_click(lambda _: encryptor.process())
start_decryption_widget.on_click(lambda _: decryptor.process())

display(
    shared_key_part_widget,
    Markdown("### Encryption"),
    decrypted_widget,
    generate_decrypted_widget,
    public_key_part_widget,
    start_encryption_widget,
    encryption_output_widget,
    Markdown("### Decryption"),
    encrypted_widget,
    generate_encrypted_widget,
    private_key_part_widget,
    start_decryption_widget,
    decryption_output_widget
);